In [1]:
import numpy as np
import pandas as pd
import datetime
import copy
import time
import os
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import operator

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
from datetime import timedelta

tqdm.pandas()

In [2]:
# Edit to point to your MIMIC directory.
dataDirStr = '/Users/gmessier/data/mimic-1.4/'

In [3]:
noteevents_df = pd.read_csv(dataDirStr + "NOTEEVENTS.csv")
noteevents_df.columns = noteevents_df.columns.str.lower()
noteevents_df

/Users/gmessier/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


`NOTEEVENTS` contains all notes for patients. `subject_id`, `hadm_id`, and `cgid` are linked to the patients, admissions and caregiver tables respectively.

In [4]:
print(f"There are {noteevents_df.subject_id.nunique()} patients who have notes recorded during their stay in the ICU")

There are 46146 patients who have notes recorded during their stay in the ICU


`category` and `description` are both categorical data with a possible 15 categories, these define the type of note recorded.

In [5]:
c = noteevents_df.category.value_counts()[:5]
p = noteevents_df.category.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
Nursing/other,822497,39.48
Radiology,522279,25.07
Nursing,223556,10.73
ECG,209051,10.04
Physician,141624,6.80


In [6]:
c = noteevents_df.description.value_counts()[:5]
p = noteevents_df.description.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
Report,1132519,54.36
Nursing Progress Note,191836,9.21
CHEST (PORTABLE AP),169270,8.13
Physician Resident Progress Note,62698,3.01
CHEST (PA & LAT),43158,2.07


In [7]:
noteevents_df.iserror.value_counts(dropna=False)

NaN    2082294
1.0        886
Name: iserror, dtype: int64